In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from os import listdir
from os.path import join
from sklearn import metrics
init_notebook_mode()

In [ ]:
def generate_data(mu=0,sigma=0.3):
    t=np.arange(0,10,0.01)
    s = np.random.normal(mu, sigma, 1000)
    x=(np.sin(2*np.pi*t)+s)
    return x
xx=generate_data()

In [ ]:
def visualisation():
    plt.plot(y_test,label='Actual')
    plt.plot(y_pred,label='Forecast')
    plt.legend(loc="upper left")
    return plt.show

# NEW

In [ ]:
def data_preparation(data,time_wind=2,output=1,nb_var=1,batch_size=1):
    data_train=data[:901]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=(x_train[:(len(x_train)-(len(x_train)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_train=data_train[time_wind:len(data_train)]#+3]
    y_batches=(y_train[:(len(y_train)-(len(y_train)%batch_size))]).reshape(-1,output)
    data_test=data[902:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=(x_t[:(len(x_t)-(len(x_t)%batch_size))]).reshape(-1,time_wind,nb_var)
    #x_test=x_t.reshape(-1,batch_size,time_wind)
    y_t=data_test[time_wind:len(data_test)]
    y_test=(y_t[:(len(y_t)-(len(y_t)%batch_size))]).reshape(-1,output)
    return x_batches, y_batches, x_test, y_test


In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(xx)

In [ ]:
print(np.shape(x_batches),np.shape(y_batches),np.shape(x_test),np.shape(y_test))

In [ ]:
def run_graph(epochs,time_wind,hidden=10,nb_var=1,output=1,lr=0.01):

    from tensorflow.contrib import rnn
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,time_wind,nb_var])
    Y=tf.placeholder(tf.float32,[None,output])
 
    tf.set_random_seed(1)
    x=tf.unstack(X,time_wind,1)
    
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden)
    rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)

    stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output)
    #outputs=tf.reshape(stacked_outputs,[-1,90,1])
    loss=tf.reduce_mean(tf.squared_difference(stacked_outputs, Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=lr)
    training_op=optimizer.minimize(loss)
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            if ep % 100 == 0:
                mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tMSE:",mse)
        y_pred=sess.run(stacked_outputs,feed_dict={X:x_test})
        mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
        print(mse)
        print(mse_test)
    sess.close() 
    return mse_test, y_pred

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,1,hidden=10,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))


In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,1,hidden=10,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,1,hidden=10,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    mse_test,y_pred=run_graph(1000,1,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,1,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),2)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,2,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),2)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,2,hidden=10,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),2)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,2,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),3)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,3,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,4,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),5)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,5,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,6,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),7)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,7,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),8)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,8,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),8)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,8,hidden=1,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,110,5):
    mse=[]
    np.random.seed(j)
    #tf.set_random_seed(j)
    for i in range (10):
        #np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
        #tf.set_random_seed(i)
        mse_test,y_pred=run_graph(1000,6,hidden=j,lr=0.1)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

In [ ]:
plt.plot(mse_var)

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,6,hidden=20,lr=0.1)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,6,hidden=20,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,6,hidden=20,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,6,hidden=20,lr=0.5)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,6,hidden=10,lr=0.5)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,6,hidden=50,lr=0.5)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,1,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,1,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,1,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(3000,1,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(5000,1,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(5000,6,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(10000,6,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(10500,6,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(11000,6,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(12000,6,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15000,6,hidden=1,lr=0.0005)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15500,6,hidden=1,lr=0.00045)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15500,6,hidden=1,lr=0.00044)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15500,6,hidden=1,lr=0.00043)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15700,6,hidden=1,lr=0.00043)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15700,6,hidden=2,lr=0.00043)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,110,5):
    mse=[]
    np.random.seed(j)
    #tf.set_random_seed(j)
    for i in range (10):
        #np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
        #tf.set_random_seed(i)
        mse_test,y_pred=run_graph(1000,1,hidden=j,lr=0.1)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

In [ ]:
plt.plot(mse_var)

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,6,hidden=10,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,6,hidden=10,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(3500,6,hidden=10,lr=0.0004)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(3500,6,hidden=10,lr=0.0001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(3500,6,hidden=20,lr=0.0001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(6000,6,hidden=20,lr=0.00007)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
def generate_new_data(mu=0, sigma=0.3):
    t=np.arange(0,10,0.01)
    t1=np.arange(0,20,0.02)
    s = np.random.normal(mu, sigma, 1000)
    x=(np.cos(2*np.pi*t1)*(np.sin(2*np.pi*t))+s)
    return x

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15500,6,hidden=1,lr=0.00044)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),1)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,1,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),2)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,2,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),3)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,3,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,4,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),5)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,5,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,6,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),7)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(1000,7,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,4,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(4000,4,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(4500,4,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(16000,4,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(20000,4,hidden=1,lr=0.001)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
 mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(15700,6,hidden=2,lr=0.00043)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))

In [ ]:
plt.plot(y_test,'o')
plt.plot(y_pred,'*')

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_new_data(),4)
    #tf.set_random_seed(i)
    mse_test,y_pred=run_graph(2000,4,hidden=1,lr=0.01)
    mse.append(mse_test)
visualisation()
print("Mean of MSE for the model:{}".format(np.mean(mse)))
print("Variance of MSE for the model:{}".format(np.var(mse)))